## Testing behaviour on concept drift

In [1]:
# import the functions needed for validate and comparsion

import numpy as np
import cupy as cp
import pandas as pd
from timeit import default_timer as timer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from skmultiflow.data import FileStream, ConceptDriftStream, RandomRBFGenerator, RandomRBFGeneratorDrift
from skmultiflow.neural_networks import PerceptronMask
from skmultiflow.trees import HoeffdingTreeClassifier



# using plotly for plots
#import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
# stability measurment as proposed in "Measurment the Stability of Feature Selection"

def pearson_stability_ij(arr1,arr2):
    d = len(arr1)
    k_i = np.sum(arr1)
    k_j = np.sum(arr2)
    x_hat_i = k_i / d
    x_hat_j = k_j / d
    arr1 = arr1 - x_hat_i
    arr2 = arr2 - x_hat_j
    dividend = 1/d * np.sum(arr1*arr2)
    divisor = np.sqrt(1/d*np.sum(arr1**2))*np.sqrt(1/d*np.sum(arr2**2))
    return dividend/divisor

def stability_factor(selected_ftrs):
   M = len(selected_ftrs)
   sum_stabilities = 0
   for i in range(M):
       for j in range(i+1, M):
           sum_stabilities += pearson_stability_ij(selected_ftrs[i], selected_ftrs[j])
   return 1/(M*(M-1))*sum_stabilities * 2   

In [3]:
# import algorithms
from fires import FIRES
from ofs import MC_OFS
from ofssgr import MC_OFSSGD
from fsds import StreamFeatWeight

### Streaming Data Generation with Concept Drift


Extra Notebook for this data, cause it doesn't have test data and the stream is unlimited 

In [ ]:
# stream generator to test how algorithms perform on data with concept drift
stream = RandomRBFGeneratorDrift(model_random_state=99, sample_random_state = 50,
 n_classes = 6, n_features = 200, n_centroids = 50, change_speed=0.05,
 num_drift_centroids=25)

n_selected_ftr = 50
batch_size = 100
n_window = 10

In [13]:
# stream 1
drift_stream = RandomRBFGenerator(model_random_state=42, sample_random_state=43, n_classes=10, n_features=100)
start_stream = RandomRBFGenerator(model_random_state=52, sample_random_state=53, n_classes=10, n_features=100, n_centroids=30)

stream = ConceptDriftStream(stream=start_stream, drift_stream=drift_stream, position=10000, width=1)

dataset_name = "Concept_drift_1"
n_selected_ftr = 50
n_window = 10
batch_size = 100

In [25]:
# stream 2
drift_stream = RandomRBFGenerator(model_random_state=20, sample_random_state=21, n_classes=10, n_features=300)
start_stream = RandomRBFGenerator(model_random_state=66, sample_random_state=67, n_classes=10, n_features=300)

stream = ConceptDriftStream(stream=start_stream, drift_stream=drift_stream, position=10000, width=1000)

dataset_name = "Concept_drift_2"
n_selected_ftr = 50
n_window = 10
batch_size = 100

### FIRES model

In [26]:
stream.restart()
predictor = PerceptronMask()
x,y = stream.next_sample(batch_size=batch_size)
predictor.partial_fit(x,y, stream.target_values)

PerceptronMask(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
               fit_intercept=True, max_iter=1000, n_iter_no_change=5,
               n_jobs=None, penalty=None, random_state=0, shuffle=True,
               tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False)

In [27]:
fires_model = FIRES(n_total_ftr=stream.n_features,
                    target_values=stream.target_values,
                    mu_init=0,
                    sigma_init=1,
                    model='softmax')        

In [28]:
fires_accuracy = []
fires_f1 = []


fires_selected_ftrs = []
fires_stability = []

start_time_all = timer()
# stream has unlimeted samples so set for loop
for i in range(200):
    # Load a new sample
    x, y = stream.next_sample(batch_size=batch_size)
    # Select features
    
    ftr_weights = fires_model.weigh_features(x,y)
    ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]

    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, ftr_selection] = x[:, ftr_selection]

    # stability test
    ftr_array = np.zeros(stream.n_features)
    ftr_array[ftr_selection] = 1
    fires_selected_ftrs.append(ftr_array)

    if len(fires_selected_ftrs) >= 10:
        stability = stability_factor(fires_selected_ftrs[-10:])
        fires_stability.append(stability)


    # Test
    y_pred = predictor.predict(x_reduced)
    
    fires_accuracy.append(accuracy_score(y, y_pred))
    fires_f1.append(f1_score(y, y_pred, average="weighted", labels=stream.target_values))


    # Train
    predictor.partial_fit(x_reduced, y)

# Restart the FileStream
end_time_all = timer()
fires_run_time = timer() - start_time_all
print("The whole FIRES run took {}".format(fires_run_time))

fires_moving_average = pd.Series(fires_accuracy).rolling(window=n_window).mean().iloc[n_window-1:].values
fires_f1 = pd.Series(fires_f1).rolling(window=n_window).mean().iloc[n_window-1:].values

The whole FIRES run took 20.784268971000074


In [29]:
fig = px.line(y=fires_moving_average, title="Moving average on accuracy", labels={"index":"batches", "value":"accuracy"})
fig.show()

In [30]:
fig = px.line(y=fires_stability, title="Stability", labels={"index":"batches", "value":"stability"})
fig.show()

In [ ]:
stream.get_data_info()


